In [1]:
import os

import pandas as pd
import numpy as np
import librosa

import glob

from PIL import Image
from tqdm import tqdm

from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
np.pad([1,2,3],10,"constant")

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

## Preprocess all files
Location: work/image/[sub]/[file].png

In [2]:
def get_files(type):
    src_files = glob.glob(f"input/{type}/*.wav")
    dst_files = [os.path.split(file) for file in src_files]
    dst_files = [(path.split("/")+file.split(".")) for (path, file) in dst_files]
    dst_files = [(os.path.join("work",f[1],f"{f[2]}/ft.png"),
                os.path.join("work",f[1],f"{f[2]}/mel.png"),
                os.path.join("work",f[1],f"{f[2]}/cqt.png")) for f in dst_files]
    audio_files = [(src_files[i],dst_files[i]) for i in range(len(src_files))]
    
    return audio_files

audio_files = get_files("trn_curated")
audio_files[0]

('input/trn_curated/ffe2178b.wav',
 ('work/trn_curated/ffe2178b/ft.png',
  'work/trn_curated/ffe2178b/mel.png',
  'work/trn_curated/ffe2178b/cqt.png'))

In [3]:
def to_image(file):
    # load file
    y, sr = librosa.load(file)

    # STFT
    D = librosa.stft(y)
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

    # Mel Spectrum
    S = librosa.feature.melspectrogram(y=y, sr=sr)
    S_db_mel = librosa.amplitude_to_db(S, ref=np.max)

    # CQT
    C = librosa.cqt(y, sr=sr)
    C_db = librosa.amplitude_to_db(abs(C), ref=np.max)

    # Normalization
    S_db = np.clip(S_db, -80, 0)
    S_db = 65535 * (S_db + 80) / 80.0

    S_db_mel = np.clip(S_db_mel, -80, 0)
    S_db_mel = 65535 * (S_db_mel + 80) / 80.0

    C_db = np.clip(C_db, -80, 0)
    C_db = 65535 * (C_db + 80) / 80.0

    # to image
    S_db = Image.fromarray(S_db.astype(np.uint16))
    S_db_mel = Image.fromarray(S_db_mel.astype(np.uint16))
    C_db = Image.fromarray(C_db.astype(np.uint16))

    return S_db, S_db_mel, C_db

In [4]:
ft,mel,cqt = to_image(audio_files[4][0])
ft.save("test.png")